In [ ]:
#Creating empty folder 
!mkdir data
%cd data

/content/data


In [ ]:
#Downlaod dataset
!wget --no-check-certificate "https://onedrive.live.com/download?cid=A86CBC7F31A1C06B&resid=A86CBC7F31A1C06B%21121&authkey=AMUUk0Np4tqH3n4" -O ts.zip
!unzip ts.zip
!rm -f ts.zip
% cd ..

## Pre-process image: Data augmentation

In [ ]:
!pip install clodsa

     |████████████████████████████████| 5.7 MB 7.2 MB/s 
     |████████████████████████████████| 276 kB 50.5 MB/s 
  Created wheel for clodsa: filename=clodsa-1.2.47-py2.py3-none-any.whl size=74310 sha256=5e904d5611be8b10c9cf75d63cd0a29ee61f2da30df97280b722698b849bf433
  Stored in directory: /root/.cache/pip/wheels/05/ff/0a/0e6e14c2a68d6869a010e979b8fd9d669aaeaa2d8b29de394f
  Created wheel for commentjson: filename=commentjson-0.9.0-py3-none-any.whl size=12092 sha256=fd9e7ed154b174a3413448f53ed814dfcc2b452e909b34c2b7011209687a410a
  Stored in directory: /root/.cache/pip/wheels/eb/bb/07/25a7f0718ee3fe137384011b8e56070f91cf950ee6047c287f
  Created wheel for lark-parser: filename=lark_parser-0.7.8-py2.py3-none-any.whl size=62527 sha256=37fd820c2c013388a8156a663e703ecc643b8637fc13c7e76f93c50f52e37d44
  Stored in directory: /root/.cache/pip/wheels/92/e3/af/1dc0fdca93232d700ac176af6554cf22b85f3d7e8aeee5ac08
Successfully built clodsa commentjson lark-parser


In [ ]:
from matplotlib import pyplot as plt
from clodsa.augmentors.augmentorFactory import createAugmentor
from clodsa.transformers.transformerFactory import transformerGenerator
from clodsa.techniques.techniqueFactory import createTechnique
import xml.etree.ElementTree as ET
import cv2
%matplotlib inline

In [ ]:
PROBLEM = "detection" #Detection problem
ANNOTATION_MODE = "yolo" #YOLO mode
INPUT_PATH = "data/ts" #image path
GENERATION_MODE = "linear" #Applied to all images of the dataset
OUTPUT_MODE = "yolo"
OUTPUT_PATH= "data/ts_aug" #output path

#Create augmentator object
augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH})

#Define transformer generator
transformer = transformerGenerator(PROBLEM)

In [ ]:
#Horizontal and Vertical flip
hvFlip = createTechnique("flip",{"flip":-1})
augmentor.addTransformer(transformer(hvFlip))

#Rotation
rotate = createTechnique("rotate", {"angle" : 90})
augmentor.addTransformer(transformer(rotate))

#Average blurring
avgBlur =  createTechnique("average_blurring", {"kernel" : 5})
augmentor.addTransformer(transformer(avgBlur))

#Raise Hue
hue = createTechnique("raise_hue", {"power" : 0.9})
augmentor.addTransformer(transformer(hue))

#To keep original image
none = createTechnique("none",{})
augmentor.addTransformer(transformer(none))

In [ ]:
#Applying the augmentation process
augmentor.applyAugmentation()

In [ ]:
#Check output folder

print("Number of images in the folder")
!ls -1 data/ts_aug/*.jpg | wc -l
print("Number of annotations in the folder")
!ls -1 data/ts_aug/*.txt | wc -l

Number of images in the folder
3705
Number of annotations in the folder
3705


## Pre-process image:Binarise image

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob
import numpy as np
from PIL import Image
import os

def read_label(image_path):
  file_name = image_path.replace('.jpg', '.txt')
  with open(file_name, 'rt') as file:
    print(os.path.basename(file_name) + ': \n' + file.read())

In [ ]:
location = "data/ts_aug"
thresh=128
maxval = 255
i=0

for pathAndFilename in glob.iglob(os.path.join(location, "*.jpg")):
    title, ext = os.path.splitext(os.path.basename(pathAndFilename))
    image_path="data/ts_aug/"+title+ext
    image=plt.imread(image_path)
    im_gray = np.array(Image.open(image_path).convert('L'))
    #delete image
    !rm $image_path
    #Binarize image
    im_bin = (im_gray > thresh) * maxval
    Image.fromarray(np.uint8(im_bin)).save(image_path)
print("Done")

Done


## Pre-process image: Reconstruction of train.txt and test.txt

In [ ]:
#Delete old train.txt and test.txt
!rm data/train.txt
!rm data/test.txt

#Create new empty file
file_train = open('data/train.txt', 'w')
file_test = open('data/test.txt', 'w')

In [ ]:
img_path="data/ts_aug"

#10% of the image for testing
percentage=10

In [ ]:
counter = 1
index_test = round(100 / percentage)
for pathAndFilename in glob.iglob(os.path.join(img_path, "*.jpg")):
    title, ext = os.path.splitext(os.path.basename(pathAndFilename))

    if counter == index_test:
        counter = 1
        file_test.write(img_path + "/" + title + '.jpg' + "\n")
    else:
        file_train.write(img_path + "/" + title + '.jpg' + "\n")
        counter = counter + 1

## Save image into a zip file in the drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls drive/MyDrive/FYP

Makefile


In [ ]:
#Zip the data file and save it in the google drive
!zip -r drive/MyDrive/FYP/data.zip data

Streaming output truncated to the last 5000 lines.
  adding: data/ts_aug/350_3_00857.txt (deflated 39%)
  adding: data/ts_aug/223_4_00395.jpg (deflated 25%)
  adding: data/ts_aug/23_2_00054.txt (deflated 56%)
  adding: data/ts_aug/249_4_00844.jpg (deflated 10%)
  adding: data/ts_aug/216_4_00788.txt (deflated 21%)
  adding: data/ts_aug/613_4_00167.jpg (deflated 6%)
  adding: data/ts_aug/407_1_00784.jpg (deflated 39%)
  adding: data/ts_aug/377_0_00119.jpg (deflated 11%)
  adding: data/ts_aug/290_2_00172.txt (deflated 46%)
  adding: data/ts_aug/512_0_00112.txt (deflated 38%)
  adding: data/ts_aug/533_3_00195.txt (deflated 42%)
  adding: data/ts_aug/574_3_00453.jpg (deflated 9%)
  adding: data/ts_aug/194_1_00033.jpg (deflated 3%)
  adding: data/ts_aug/701_2_00125.txt (deflated 20%)
  adding: data/ts_aug/612_4_00220.txt (deflated 19%)
  adding: data/ts_aug/322_2_00653.txt (deflated 40%)
  adding: data/ts_aug/155_3_00140.jpg (deflated 9%)
  adding: data/ts_aug/13_2_00240.jpg (deflated 17%)
 